# Connected Four - Projeto de Inteligência Artificial

Realizado pelos alunos:
- Rita Moreira (202303885);
- Pedro Gilvaia (202306975);
- Gonçalo Correia (202208527).

## 1. Introdução

O objetivo deste projeto é implementar o algoritmo Monte Carlo Tree Search (MCTS) e uma árvore de decisão para cada dataset dado (iris.csv e o dataset gerado usando o MCTS), usando o algoritmo ID3. Usando esses procedimentos, será então criado um programa capaz de jogar o Quatro em Linha (Connected Four) contra um humano.
Este notebook documenta todo o processo de implementação do jogo, incluindo: 
- explicações das decisões tomadas em cada etapa;
- detalhes da geração de dados com MCTS;
- Treino da árvore de decisão com ID3;
- Integração dos algoritmos no jogo;
- Discussão de resultados.

## 2. Descrição do Problema

O Quatro em Linha é um jogo de estratégia de dois jogadores. São usadas 42 peças, 21 para cada jogador, e um tabuleiro vertical com 7 colunas e 6 filas. A cada jogada, o jogador atual "deixa cair" a sua peça numa coluna à sua escolha, desde que não esteja cheia. Esta cai na linha disponível. Um jogador ganha quando 4 das suas peças formarem uma linha horizontal, vertical ou diagonal consecutiva.

### 2.1. Implementação do Jogo

Abaixo segue-se a implementação das funcionalidades do jogo (connected_four.py). Mais tarde, será apresentada a implementação do jogo em si (game.py), que utiliza todos os códigos trabalhados no projeto (connected_four.py, mcts.py e decision_tree_builder.py).

#### __connected_four.py__

Importações necessárias:

In [ ]:
from copy import deepcopy
import math

A classe `GameMeta` funciona como um container de constantes e parâmetros globais que descrevem as regras básicas e convenções do jogo Connect Four na sua implementação. 

In [ ]:
class GameMeta:
    # dicionário que mapeia nomes de jogador para valores usados no jogo
    PLAYERS = {'none': 0, 'one': 1, 'two': 2} 
    # dicionário que mapeia resultados possíveis de uma partida
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    # constante que representa "infinito" em pontuações ou limites
    INF = float('inf')
    # definição do tamanho do tabuleiro (ROWS x COLS)
    ROWS = 6
    COLS = 7

A classe `MCTSMeta` define o coeficiente de exploração (c). Tem como papel balancear o teste de ramos menos visitados para descobrir se possuem bons resultados (exploit) e a escolha do ramo com maior valor médio $\frac{Q}{N}$. Um c grande amplia a exploração de nós (explore) e um c pequeno prioriza o valor $\frac{Q}{N}$ (exploit).

In [ ]:
class MCTSMeta:
    EXPLORATION = 1.4

A classe `ConnectState` representa o estado de um jogo. A função `__init__` é um construtor da classe, e inicializa o tabuleiro, o jogador atual, a altura de cada coluna e a última jogada feita.

Possui atributos como:
- `board`: matriz 6x7 com valores 0 (vazio), 1(jogador 1) e 2 (jogador 2);
- `to_play`: identificador do jogador que jogará a seguir;
- `height`: lista de índices da próxima linha livre em cada coluna
- `last_played`: par [linha, coluna] da última jogada.

In [ ]:
class ConnectState:
    def __init__(self):
        # Inicializa o tabuleiro com zeros (vazio)
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        # Inicializa o jogador atual como 'one' (1)
        self.to_play = GameMeta.PLAYERS['one']
        # Inicializa a altura de cada coluna como o número de linhas menos 1 (posição inicial)
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        # Inicializa a lista de ações jogadas como nula
        self.last_played = []


A função `get_board` retorna uma cópia do tabuleiro atual.

In [ ]:
def get_board(self):
        return deepcopy(self.board)

A função `move` executa um movimento numa coluna específica e atualiza o estado de jogo. Se a coluna estiver cheia, surge um `ValueError`. Possui como parâmetro a coluna onde o jogador quer jogar.

In [ ]:
def move(self, col):
        # Verifica se a coluna está cheia
        if self.height[col] < 0:
            raise ValueError(f"Coluna {col} já está cheia!")  # Ou apenas retorna False ou trata o erro de forma apropriada
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

A função `get_legal_moves` retorna uma lista dos índices das colunas onde ainda é possível jogar, isto é, que não estejam cheias. 

In [ ]:
def get_legal_moves(self):
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

A função `check_win` verifica se o jogador atual venceu. Se sim, retorna o código do mesmo (1 ou 2, dependendo do jogador). Se não houver vitória, retorna 0.

In [ ]:
def check_win(self):
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0

A função `check_win_from` verifica se há "quatro em linha" a partir da posição (row, col) em todas as direções: horizontal, vertical, diagonal principal e diagonal secundária. Se sim, retorna True. Caso contrário, retorna False. Tem como parâmetros a linha (`row`) e a coluna (`col`) da última jogada.

In [ ]:
def check_win_from(self, row, col):
        player = self.board[row][col]
        consecutive = 1
        # Check horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # Check vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # Check diagonal
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # Check anti-diagonal
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False


A função `game_over` verifica se o jogo acabou, retornando um valor booleano consoante o resultado.

In [ ]:
def game_over(self):
        return self.check_win() or len(self.get_legal_moves()) == 0

A função `get_outcome` retorna o código de resultado (outcome) da partida. Se for vitória retorna 1 ou 2, dependendo do jogador vencedor. Se for empate retorna 3. 

In [ ]:
def get_outcome(self):
        if len(self.get_legal_moves()) == 0 and self.check_win() == 0:
            return GameMeta.OUTCOMES['draw']

        return GameMeta.OUTCOMES['one'] if self.check_win() == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']


Por último, a função `print` exibe o tabuleiro no terminal e substitui o número que está na matriz (tabuleiro) pela peça de jogo ou espaço vazio:
- ' ' (espaço) substitui o 0;
- X substitui o 1;
- O substitui o 2.

In [ ]:

def print(self):
    print('=============================')

    for row in range(GameMeta.ROWS):
        for col in range(GameMeta.COLS):
            print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
        print('|')

    print('=============================')

## 3. Metodologia e sua implementação

### 3.1. Monte Carlo Tree Search (MCTS)

O MCTS é um algoritmo de busca adversarial que leva em consideração as alterações de estado causadas por um adversário nas etapas subsequentes. A sua implementação utiliza o critério Upper Confidence Bound (UCT) para avaliar cada ramo de uma árvore, de fórmula:
$$
UCT = \frac{Q}{N} + c\sqrt{\frac{In{N_{parent}}}{N}}
$$
O MCTS tem quatro fases:
- Seleção: Começa na raiz e, em cada nó, escolhe o filho que maximiza o UCT até atingir um nó ainda não totalmente expandido;
- Expansão:  A partir do nó selecionado, gera um dos estados-filho ainda não representados na árvore, adicionando-o como novo nó;
- Simulação: A partir desse nó recém-criado, executa um jogo até ao fim (playout), usando uma política aleatória ou heurística, obtendo um resultado (vitória/derrota);
- Retropropagação (Backpropagation): Propaga o resultado da simulação de volta à raiz, atualizando em cada nó visitado. Nesses nós, a contagem de visitas aumenta 1 valor e contagem de vitórias aumenta o valor do resultado (0 se perdeu, 1 se ganhou).


#### __mcts.py__

Importações necessárias:

In [ ]:
import random
import time
import math
from copy import deepcopy

from connected_four import ConnectState, GameMeta, MCTSMeta

A classe Node representa um nó de uma árvore de pesquisa Monte Carlo. A função `__init__` inicializa um nó e seus atributos:
- `move`: jogada que levou ao nó atual;
- `parent`: nó pai da árvore (a razi tem parent=None);
- `N`: número de visitas;
- `Q`: Soma das recompensas (vitórias) obtidas deste nó;
- `children`: mapeia jogadas para nós-filho;
- `outcome`: estado de resultado no nó (0=nenhum, 1=jogador1, 2=jogador2).

In [ ]:
class Node:
    def __init__(self, move, parent):
        self.move = move
        self.parent = parent
        self.N = 0
        self.Q = 0
        self.children = {}
        self.outcome = GameMeta.PLAYERS['none']


A função `add_children` adiciona nós-filho a partir de um dicionário (parâmetro).

In [ ]:
def add_children(self, children: dict) -> None:
        for child in children:
            self.children[child.move] = child

A função `get_exploration` calcula dinamicamente e retorna o coeficiente da exploração `c` para UCT. Baseia-se no número de visitas do nó pai e o valor diminui consoante a raiz quadrada desse número, levando a uma exploração maior no início e mais leve no final.

In [ ]:
def get_exploration(self) -> float:
        c0 = MCTSMeta.EXPLORATION #1.4
        if self.parent is not None:
            root_visits = self.parent.N 
            alpha = 0.2
            return c0 / (1 +  alpha * math.log(1 + root_visits))
        

A função `value` calcula e retorna o valor UCT do nó para a seleção. Utiliza como parâmetro o coeficiente inicial de exploração: 1.4.

In [ ]:

def value(self, explore: float = MCTSMeta.EXPLORATION):
    if self.N == 0:
        return GameMeta.INF # forçar exploração de nós não visitados
    else: #dynamic c
        c_now = self.get_exploration()
        return self.Q / self.N + c_now * math.sqrt(math.log(self.parent.N) / self.N)

A classe MCTS implementa o algoritmo para o Connected Four. A função `__init__` inicializa o MCTS com um estado inicial, que provem da classe ConnectedState.  Tem como atributos:
- `root_state`: estado de jogo na raiz;
- `root`: nó raiz da árvore de busca;
- `run_time`: tempo gasto na última pesquisa (segundos de CPU);
- `num_rollouts`: número de simulações executadas na última pesquisa.

In [ ]:
class MCTS:
    def __init__(self, state=ConnectState()):
        self.root_state = deepcopy(state)
        self.root = Node(None, None)
        self.run_time = 0
        self.node_count = 0
        self.num_rollouts = 0

A primeira fase do MCTS é a seleção, representada pela função `select_node`. Esta seleciona um nó a ser expandido com base no UCT, até que um nó não visitado seja encontrado ou o jogo acabe. Retorna um tuplo, com o nó selecionado e o estado de jogo.

In [ ]:
def select_node(self) -> tuple:
        node = self.root
        state = deepcopy(self.root_state)

        # Se o nó não tiver filhos, expande-o
        while len(node.children) != 0:
            # escolher o filho com maior valor UCT
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            max_nodes = [n for n in children if n.value() == max_value]

            node = random.choice(max_nodes)
            state.move(node.move)

            if node.N == 0:
                return node, state
        # se o nó não for terminal, expande-o
        if self.expand(node, state):
            node = random.choice(list(node.children.values()))
            state.move(node.move)

        return node, state

A segunda fase é a expansão. A função `expand` cria filhos para o nó pai com base nas jogadas disponíveis. Se o estado for terminal, i.e., não houver expansão possível, retorna False. Tem como parâmetros:
- `parent` (Nó): nó pai a ser expandido;
- `state` (ConnectState): estado atual do jogo.

In [ ]:
def expand(self, parent: Node, state: ConnectState) -> bool:
        if state.game_over():
            return False

        children = [Node(move, parent) for move in state.get_legal_moves()]
        parent.add_children(children)

        return True

A terceira fase é a de simulação. A função correspondente, `roll_out`, joga aleatoriamente até ao fim do jogo e retorna o resultado do mesmo. Utiliza como argumento o estado do jogo.

In [ ]:
def roll_out(self, state: ConnectState) -> int:
        while not state.game_over():
            moves = state.get_legal_moves()
            player = state.to_play
            opp = 3 - player
            # Vitória imediata
            for m in moves:
                s2 = deepcopy(state)
                s2.move(m)
                if s2.game_over() and s2.get_outcome() == player:
                    state.move(m)
                    break
            else:
                # Bloqueio de ameaças adversárias
                blocked = False
                for m in moves:
                    s2 = deepcopy(state)
                    s2.move(m)
                    for m2 in s2.get_legal_moves():
                        s3 = deepcopy(s2)
                        s3.move(m2)
                        if s3.game_over() and s3.get_outcome() == opp:
                            state.move(m)
                            blocked = True
                            break
                    if blocked:
                        break
                if not blocked:
                    state.move(random.choice(moves))

        return state.get_outcome()

A última fase é a de Backpropagation (retropropagação). Tem como função `back_propagate`, que atualiza os valores de `N` e `Q` dos nós visitados. Tem como parâmetros:
- `node` (Nó): nó onde terminou a simulação;
- `turn` (int): jogador que fez a última jogada;
- `outcome` (int): resultado do jogo (1,2 ou 3).

In [ ]:
def back_propagate(self, node: Node, turn: int, outcome: int) -> None:
        #recompensa relativa ao jogador 'turn'
        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.Q += reward
            node = node.parent
            if outcome == GameMeta.OUTCOMES['draw']:
                reward = 0
            else:
                reward = 1 - reward

Para executar simulações do MCTS por um tempo limitado é usada a função `search`. Tem como parâmetro o `time_limit`, que guarda o temlo máximo de CPU escolhido para as simulações.

In [ ]:
def search(self, time_limit: int):
        start_time = time.process_time()

        num_rollouts = 0
        while time.process_time() - start_time < time_limit:
            node, state = self.select_node()
            outcome = self.roll_out(state)
            self.back_propagate(node, state.to_play, outcome)
            num_rollouts += 1

        run_time = time.process_time() - start_time
        self.run_time = run_time
        self.num_rollouts = num_rollouts

É utilizada a função `best_move` para determinar qual é a melhor jogada de momento, com base em `N`, a partir da raiz. Se o jogo já não tiver acabado, retorna a coluna escolhida. Se sim, retorna -1.

In [ ]:
def best_move(self):
        if self.root_state.game_over():
            return -1

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        best_child = random.choice(max_nodes)

        return best_child.move

Para a raiz ser atualizada, é usada a função `move`, de parâmetro a coluna onde o jogador jogou (`move`).

In [ ]:
def move(self, move):
        if move in self.root.children:
            self.root_state.move(move)
            self.root = self.root.children[move]
            return

        self.root_state.move(move)
        self.root = Node(None, None)

Por último, a função `statistics` retorna as estatísticas da última pesquisa:
- o número de simulações feitas;
- o tempo de CPU gasto nas mesmas.

In [ ]:
def statistics(self) -> tuple:
    return self.num_rollouts, self.run_time

### 3.2. Geração do dataset

Usando o algoritmo MCTS, foi gerado um dataset para treinar uma árvore de decisão, abordada de seguida. Este código simula múltiplas partidas de Connect Four usando o MCTS para gerar um conjunto de exemplos (estado, movimento) que servirão de treino para a árvore de decisão ID3. Cada linha do dataset contém 42 atributos (o tabuleiro achatado) seguidos da coluna escolhida pelo MCTS como 'move'.

Importações necessárias:

In [ ]:
import random
import numpy as np
from mcts import MCTS
from connected_four import ConnectState, GameMeta, MCTSMeta

A função `save_dataset` recebe a lista de tuplos `dataset` e o nome do ficheiro de saída, e salva-a como um ficheiro CSV com cabeçalho. O CSV gerado terá colunas 's0',...,'s41' e 'move'. O CSV gerado terá colunas 's0'...'s41' e 'move'.

In [ ]:
def save_dataset(dataset, filename='mcts_dataset.csv'):
    with open(filename, 'w') as f:
        # Cabeçalho com 42 features e o rótulo move
        f.write(','.join([f's{i}' for i in range(42)]) + ',move\n')
        for state, move in dataset:
            f.write(','.join(map(str, state)) + f',{move}\n')

A função `generate_mcts_dataset` gera um dataset de pares (estados, movimento) usando o MCTS. Tem como parâmetro o número de partidas a simular (`num_games`) e retorna uma lista de tuplos (`board_flat`, `move`). Cada estado `board_flat` é uma lista de 42 inteiros representando o tabuleiro, seguido de `move` (coluna escolhida pelo MCTS). O MCTS recebe um tempo de busca variável: 3 segundos por jogada inicial e, após 10 jogadas, passa a 1.5 segundos. A função salva o dataset parcialmente a cada 10 jogos.

In [ ]:
def generate_mcts_dataset(num_games=1000):
    dataset = []

    for game_idx in range(num_games):
        print(f"\n🎮 Jogo {game_idx + 1} de {num_games}")
        state = ConnectState()
        mcts = MCTS(state)
        total_moves = 0  # contador de jogadas
        mcts.search(time_limit=3)  # Tempo inicial para exploração mais profunda

        while not state.game_over():
            legal_moves = state.get_legal_moves()
            move = mcts.best_move()
            # Verifica e corrige movimento ilegal, se houver
            if move not in legal_moves:
                print(f"[!] Movimento ilegal sugerido: {move}")
                move = random.choice(legal_moves)

            try:
                 # Flatten do tabuleiro antes de aplicar a jogada
                board_flat = [cell for row in state.get_board() for cell in row]
                dataset.append((board_flat, move))
                # Aplica jogada no estado e no MCTS
                state.move(move)
                mcts.move(move)
                total_moves += 1
                # Ajuste de tempo por jogada após 10 movimentos
                if not state.game_over():
                    time_limit = 3 if total_moves < 10 else 1.5
                    mcts.search(time_limit=time_limit)

            except ValueError as e:
                print(f"[Erro] Movimento inválido: {e}")
                break

        print(f"✔️ Jogo {game_idx + 1} concluído — total de jogadas salvas: {len(dataset)}")

        # Salvamento parcial a cada 10 jogos
        if (game_idx + 1) % 10 == 0:
            save_dataset(dataset, filename='mcts_dataset_parcial.csv')
            print(f"💾 Dataset parcial salvo com {len(dataset)} jogadas.")

    return dataset

Após isso, é possível fazer execução direta do gerador e salvar o dataset final. Neste caso foram escolhidos 1000 jogos, para ter boas informações e a árvore de decisão ser robusta.

In [ ]:
# Geração e salvamento
if __name__ == "__main__":
    dataset = generate_mcts_dataset(num_games=1000)
    save_dataset(dataset)
    print(f"\n✅ Dataset final salvo com {len(dataset)} exemplos!")


### 3.3. Árvores de Decisão

Para o segundo método, foi utilizado um dataset (`iris.csv`) para testar ("warm-up") o algoritmo ID3. É um dataset mais pequeno, de contexto diferente, que tem como tarefa gerar/treinar uma árvore de decisão que, dadas quatro características (comprimento e largura da pétala, comprimento e largura da sépala), determine a que classe cada planta pertence (Iris setosa, Iris virginica ou Iris versicolor).
Após esse teste, o objetivo é gerar uma árvore de decisão associada à implementação do algoritmo MCTS. É gerado um conjunto de pares ($state_{i}, move_{i}$), em que $state_{i}$ é o estado corrente do jogo e $move_{i}$ é o próximo movimento sugerido pelo algoritmo. Com esse dataset (`mcts_dataset.csv`), é treinada uma árvore de decisão usando o algoritmo ID3 (Iterative Dichotomiser 3), de forma a que, dado um estado de jogo, a árvore escolha o próximo movimento.
Abaixo serão apresentadas os ficheiros decision_tree_builder.py, que implementa o algoritmo ID3, e `iris_test.py`, que aplica esse algoritmo no seu dataset.


#### __decision_tree_builder.py__

Importações necessárias:

In [ ]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

A classe Node representa um nó numa árvore de decisão ID3. A função `__init__` inicializa o nó da árvore e tem como atributos:
- `feature`: nome ou índice da feature usada para dividir neste nó (None se folha);
- `children`: mapeamento do valor de feature para nós-filho;
- `label`: Rótulo da classe se este nó for folha (None caso contrário).

In [ ]:
class Node:
    def __init__(self, feature=None, children=None, label=None):
        self.feature = feature
        self.children = children if children is not None else {}
        self.label = label

A função `entropy` calcula e retorna a entropia de Shannon de um vetor/série de rótulos `y` (parâmetro). A entropia pode ser calculada usando a fórmula:
$$ H(y) = -\sum_{i} p_i \cdot \log_2(p_i) $$

In [ ]:
def entropy(y):
    counts = Counter(y)
    probabilities = [count / len(y) for count in counts.values()]
    return -sum(p * np.log2(p) for p in probabilities if p > 0)

A função `information_gain` calcula e retorna o ganho de informação ao dividir X e y pela feature especificada. Tem como parâmetros:
- `X`: DataFrame de atributos;
- `y`: Série de rótulos/classes;
- `feature`: nome da coluna de X a avaliar.
Este valor pode ser, então, calculado pela fórmula:
$$ IG = H(y) - \sum_{j} \frac{|S_j|}{|S|} \cdot H(S_j) $$

In [ ]:
def information_gain(X, y, feature):
    values = X[feature].unique()
    weighted_entropy = 0

    for v in values:
        subset_y = y[X[feature] == v]
        weighted_entropy += (len(subset_y) / len(y)) * entropy(subset_y)

    return entropy(y) - weighted_entropy

A principal função para a criação de uma árvore de decisão é o algoritmo `id3` (Iterative Dichotomiser 3), que retorna o nó raiz da árvore de decisão. Tem como parâmetros:
- `X`: Dataframe de atributos;
- `y`: Série de rótulos/classes correspondente;
- `features`: Lista de colunas ainda disponíveis para divisão;
- `depth`: Profundidade atual da árvore;
- `max_depth`: Profundidade máxima permitida.

In [ ]:
def id3(X, y, features, depth=0, max_depth=15):
    # critérios de paragem: pureza, sem features ou profundidade máxima
    if len(set(y)) == 1 or len(features) == 0 or depth == max_depth:
        return Node(label=y.iloc[0])
    
    if len(features) == 0:
        most_common_label = y.mode()[0]
        return Node(label=most_common_label)
    #escolha de melhor feature
    gains = [information_gain(X, y, f) for f in features]
    best_feature = features[np.argmax(gains)]

    node = Node(feature=best_feature)
    feature_values = X[best_feature].unique()
    #recursão para cada valor da feature escolhida
    for value in feature_values:
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]

        if len(subset_y) == 0:
            # nenhum exemplo com esse valor
            most_common_label = y.mode()[0]
            child = Node(label=most_common_label)
        else:
            #continua a recursão sem a feature escolhida
            remaining_features = [f for f in features if f != best_feature]
            child = id3(subset_X, subset_y, remaining_features, depth+1, max_depth)

        node.children[value] = child

    return node

A função `majority_vote` ocorre quando existem ramos nunca vistos durante a previsão (`predict`), fazendo assim uma votação majoritária nos descendentes. Tem como parâmetro o nó atual da árvore de decisão e retorna a classe mais comum entre os filhos, ou `None` se não houver filhos.

In [ ]:
def majority_vote(node):
    labels = []
    def collect_labels(subnode):
        if subnode.label is not None:
            labels.append(subnode.label)
        else:
            for child in subnode.children.values():
                collect_labels(child)
    collect_labels(node)
    if labels:
        return Counter(labels).most_common(1)[0][0]
    else:
        return None

Por fim, a função `predict` prediz a classe de um exemplo usando a árvore de decisão. Tem como parâmetros:
- `tree`: o nó raiz da árvore;
- `sample`: a classe prevista, ou `None` se não houver previsão.

Retorna a `sample`.

In [ ]:
def predict(tree, sample):
    while tree.label is None:
        value = sample.get(tree.feature)
        if value in tree.children:
            tree = tree.children[value]
        else:
            # Valor nunca visto — retorna o valor de maior ocorrência entre os filhos
            # ou uma jogada aleatória segura
            if tree.children:
                return majority_vote(tree)
            else:
                return None
    return tree.label

Após a implementação das funções, segue-se a criação da árvore. 
Começa-se por carregar os dados discretizados do dataset gerado: 

In [ ]:
# Carrega dados discretizados
data = pd.read_csv("mcts_dataset.csv")
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
features = X.columns.tolist()

Após isso, treina-se (cria-se) a árvore de decisão usando a função `id3` e testa-se a mesma com a função `predict`, calculando a acurácia desse teste:

In [ ]:
# Treina a árvore de decisão
tree = id3(X, y, features)

# Testa a árvore de decisão
sample = X.iloc[0].to_dict()  # primeira linha como dict
predicted_label = predict(tree, sample)
actual_label = y.iloc[0]

correct = 0
for i in range(len(X)):
    sample = X.iloc[i].to_dict()
    prediction = predict(tree, sample)
    if prediction == y.iloc[i]:
        correct += 1

accuracy = correct / len(X)
# print(f"Accuracy on training data: {accuracy:.2%}")

Por último, exporta-se a árvore de decisão, usando a biblioteca `pickle`, para ser usada mais tarde no ficheiro `game.py`:

In [ ]:
# Exporta a árvore de decisão
with open("decision_tree.pkl", "wb") as f:
    pickle.dump(tree, f)

#### __iris_test.py__

Importações necessárias:

In [ ]:
import pandas as pd
import pickle
from decision_tree_builder import id3, predict, Node

É necessário, inicialmente, carregar e discretizar os dados do dataset `iris.csv`. A função `discretize` discretiza uma coluna num número fixo de bins e retorna a coluna com os valores discretizados. Tem como parâmetros:
- `col`: Coluna a discretizar;
- `n_bins`: Nr de bins (default: 3).

In [ ]:
# Carregamento e discretização dos dados do dataset iris.csv
df = pd.read_csv('iris.csv')

def discretize(col, n_bins=3):
    return pd.cut(col, bins=n_bins, labels=[f'bin{i}' for i in range(n_bins)])

# Discretiza todas as colunas, exceto a última (rótulo)
for col in df.columns[:-1]:
    df[col] = discretize(df[col])

Após isso, são preparados os atributos e rótulos, tal como a divisão dos valores em treino e teste (70% de treino e 30% de teste).

In [ ]:
# Preparação de atributos e rótulos
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
features = X.columns.tolist()

# Divisão em treino e teste (70% treino, 30% teste)
df_shuffled = df.sample(frac=1).reset_index(drop=True)
split_idx = int(len(df_shuffled) * 0.7)
train = df_shuffled.iloc[:split_idx].reset_index(drop=True)
test = df_shuffled.iloc[split_idx:].reset_index(drop=True)

X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]

Com eses valores, podemos treinar a árvore de decisão, com a função `id3` e exportá-la, caso seja necessário:

In [ ]:
# treino da árvore de decisão
tree = id3(X_train, y_train, features)

# Serialização da árvore de decisão com pickle
with open('iris_tree.pkl', 'wb') as f:
    pickle.dump(tree, f)
print("Decision tree serialized to iris_tree.pkl")

Por último, a função `print_tree` imprime a árvore de decisão de forma recursiva e tem como parâmetros:
- `node` (Nó): nó atual da árvore de decisão;
- `depth`: profundidade atual da árvore.

In [ ]:
# Função para imprimir a árvore de decisão
def print_tree(node: Node, depth=0):
    '''
    Exibe a estrutura da árvore de decisão de forma recursiva
    Parâmetros:
    node (Node) - nó atual da árvore de decisão
    depth (int) - profundidade atual da árvore
    '''
    indent = '  ' * depth
    if node.label is not None:
        print(f"{indent}→ Label: {node.label}")
    else:
        feat_name = features[node.feature] if isinstance(node.feature, int) else node.feature
        print(f"{indent}Feature: {feat_name}")
        for value, child in node.children.items():
            print(f"{indent}  If == {value}:")
            print_tree(child, depth+2)

Após isso, basta chamar essa função e, por fim, avaliar a acurácia do conjunto de teste.

In [ ]:
# Exibe a estrutura da árvore de decisão gerada
print("\nDecision Tree Structure:")
print_tree(tree)

# Avalia a acurácia no conjunto de teste
correct = 0
for idx, row in X_test.iterrows():
    sample = row.to_dict()
    pred = predict(tree, sample)
    true_label = y_test.loc[idx]
    if pred == true_label:
        correct += 1
accuracy = correct / len(X_test)
print(f"\nAccuracy: {accuracy:.2%}")

## 4. Integração no Jogo

Usando os códigos acima apresentados, é então possível recriar o jogo Quatro em Linha. Em baixo está o código final, que importa todos os anteriores.

#### __game.py__

Importações necessárias:

In [ ]:
from connected_four import ConnectState, GameMeta  # Representa o estado do jogo e a inicialização do jogo
from mcts import MCTS                   # Implementa o algoritmo MCTS
import pickle # usado para manipular a árvore de decisão
from decision_tree_builder import predict  # Importa a função de previsão da árvore de decisão

As classes `RestartGameException` e `QuitGameException` são exceções específicas para controlar o fluxo do jogo, e permitem reiniciar ou sair do mesmo de forma controlada.

In [ ]:
class RestartGameException(Exception):
    pass

class QuitGameException(Exception):
    pass

Para utilizar a árvore de decisão no jogo é necessário, primeiramente, de a carregar, usando o `pickle`:

In [ ]:
# carregamento da árvore de decisão (ficheiro .pkl)
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)

A função `safe_int_input` lê inteiros de forma a tratar de inputs diferentes como `restart` e `quit`, ou erros.

In [ ]:
#Lê inteiros com tratamento para "restart" e "quit"
def safe_int_input(prompt):
    while True:
        try:
            value = input(prompt)
            if value.lower() == 'restart':
                raise RestartGameException
            if value.lower() == 'quit':
                raise QuitGameException
            return int(value)
        except ValueError:
            print("Entrada inválida. Por favor insira um número inteiro.")
        except EOFError:
            raise

A função `safe_move_input` lê uma jogada válida, i.e., um número de 0 a 6 para associar à coluna da jogada.

In [ ]:
# Lê uma jogada válida (coluna de 0 a 6)
def safe_move_input(state, prompt="Escolha uma coluna (0-6), 'restart' ou 'quit': "):
    while True:
        try:
            user_input = input(prompt)
            if user_input.lower() == 'restart':
                raise RestartGameException
            if user_input.lower() == 'quit':
                raise QuitGameException
            move = int(user_input)
            if move in state.get_legal_moves():
                return move
            print("Movimento inválido. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Insira um número de 0 a 6.")
        except EOFError:
            raise

A função `state_to_features` converte o estado de jogo em features, ou seja, transforma o estado do tabuleiro num dicionário de atributos para alimentar a árvore de decisão.

In [ ]:
def state_to_features(state):
    # Assumindo que o estado tem uma representação interna `board`
    # e que a árvore foi treinada com features nomeadas como pos_0, pos_1, ..., pos_41
    features = {}
    for i in range(6):  # 6 linhas
        for j in range(7):  # 7 colunas
            index = i * 7 + j
            features[f"s{index}"] = state.board[i][j]

    return features

De seguida temos as funções de jogo. Existem 4 modos de jogo:
- Jogador contra Jogador: tem como função `play_player_vs_player` e parâmetro o jogador inicial;

In [ ]:
def play_player_vs_player(starting_player):
    state = ConnectState()
    state.to_play = starting_player
    while not state.game_over():
        state.print()
        print(f"Vez de {'X' if state.to_play==1 else 'O'}")
        move = safe_move_input(state)
        state.move(move)
    state.print()
    return state.get_outcome()

- Jogador contra MCTS: tem como função `play_player_vs_ai` e parâmetros a peça do jogador (`player_piece`) e o jogador inicial (`first`);

In [ ]:
def play_player_vs_ai(player_piece, first):
    state = ConnectState()
    ai_piece = 2 if player_piece == 1 else 1
    mcts = MCTS(state)
    if first == 'ai':
        state.to_play = ai_piece
        state.print()
        print("MCTS a começar...")
        mcts.search(10)
        mv = mcts.best_move()
        print(f"MCTS ({'X' if ai_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)
        mcts.move(mv)
    else:
        state.to_play = player_piece
    while not state.game_over():
        state.print()
        if state.to_play == player_piece:
            mv = safe_move_input(state)
            state.move(mv)
            mcts.move(mv)
        else:
            print("MCTS a pensar...")
            mcts.search(5)
            mv = mcts.best_move()
            print(f"MCTS ({'X' if ai_piece==1 else 'O'}) escolheu: {mv}")
            state.move(mv)
            mcts.move(mv)
    state.print()
    return state.get_outcome()

- Jogador contra Árvore, de função `play_player_vs_tree` e parâmetros a peça do jogador (`player_piece`), o jogador inicial (`first`) e a árvore de decisão carregada no início do jogo (`tree`);

In [ ]:
def play_player_vs_tree(player_piece, first, tree):
    state = ConnectState()
    tree_piece = 2 if player_piece == 1 else 1

    if first == 'tree':
        state.to_play = tree_piece
        state.print()
        features = state_to_features(state)
        mv = predict(tree, features)
        print(f"Árvore ({'X' if tree_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)
    else:
        state.to_play = player_piece

    while not state.game_over():
        state.print()
        if state.to_play == player_piece:
            mv = safe_move_input(state)
        else:
            print("Árvore a pensar...")
            features = state_to_features(state)
            mv = predict(tree, features)
            print(f"Árvore ({'X' if tree_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)

    state.print()
    return state.get_outcome()

- MCTS contra Árvore, de função `play_ai_vs_tree` e parâmetros o jogador inicial (`starting_player`) e a árvore de decisão (`tree`).

In [ ]:
def play_ai_vs_tree(starting_player, tree):
    state = ConnectState()
    mcts = MCTS(state)
    ai_piece = starting_player
    tree_piece = 2 if ai_piece == 1 else 1

    while not state.game_over():
        state.print()
        if state.to_play == ai_piece:
            print("MCTS a pensar...")
            mcts.search(5)
            mv = mcts.best_move()
            print(f"MCTS escolheu: {mv}")
        else:
            print("Árvore a pensar...")
            features = state_to_features(state)
            mv = predict(tree, features)
            print(f"Árvore escolheu: {mv}")
        state.move(mv)
        mcts.move(mv)

    state.print()
    return state.get_outcome()

Após um jogo terminar, é apresentado um menu, de função `post_game_menu`, que permite ao jogador jogar novamente, mudar de modo ou sair do jogo.

In [ ]:
# Menu pós-jogo: Play Again, Change Modes, Quit
def post_game_menu():
    print("O que deseja fazer a seguir?")
    print("1: Jogar novamente")
    print("2: Mudar modo")
    print("3: Sair")
    choice = safe_int_input("Escolha: ")
    if choice == 1:
        return 'again'
    if choice == 2:
        raise RestartGameException
    return 'quit'

A função `main` lida com os menus iniciais, com a escolha do modo (que utiliza as funções acima), do jogador inicial e da peça do jogador. Cria um "placar" com o número de vitórias/derrotas caso o jogador decida fazer vários jogos seguidos no mesmo modo. Assim, durante esse tempo, o número de vitórias e derrotas é armazenado pelo sistema e impresso no terminal.

In [ ]:
# Bloco principal

def main():
    x_wins = o_wins = draws = 0
    try:
        while True:
            print("1: Jogador vs Jogador\n2: Jogador vs MCTS\n3: Jogador vs Árvore\n4: MCTS vs Árvore")
            mode = safe_int_input("Modo: ")
            # Definições iniciais do modo
            if mode == 1:
                print("Quem começa? \n1: X  \n2: O")
                first = 1 if safe_int_input("Escolha: ")==1 else 2
                while True:
                    outcome = play_player_vs_player(first)
                    if outcome == 1: x_wins += 1
                    elif outcome == 2: o_wins += 1
                    else: draws += 1
                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException
            elif mode == 2:
                print("Quem começa? \n1: Jogador  \n2: MCTS")
                first = 'player' if safe_int_input("Escolha: ") == 1 else 'ai'
                print("Que peça prefere? \n1: X  \n2: O")
                player_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                mcts_piece = 3 - player_piece

                while True:
                    outcome = play_player_vs_ai(player_piece, first)
                    # se outcome == player_piece → jogador humano ganhou
                    if outcome == player_piece:
                        if player_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == mcts_piece:
                        # MCTS ganhou
                        if mcts_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException

            elif mode == 3:
                print("Quem começa? \n1: Jogador  \n2: Árvore")
                first = 'player' if safe_int_input("Escolha: ") == 1 else 'tree'
                print("Que peça prefere? \n1: X  \n2: O")
                player_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                tree_piece = 3 - player_piece

                while True:
                    outcome = play_player_vs_tree(player_piece, first, decision_tree)
                    if outcome == player_piece:
                        # Jogador humano ganhou
                        if player_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == tree_piece:
                        # Árvore ganhou
                        if tree_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException
         
            elif mode == 4:
                print("Quem começa? \n1: MCTS  \n2: Árvore")
                mcts_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                tree_piece = 3 - mcts_piece

                while True:
                    outcome = play_ai_vs_tree(mcts_piece, decision_tree)
                    if outcome == mcts_piece:
                        # MCTS ganhou
                        if mcts_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == tree_piece:
                        # Árvore ganhou
                        if tree_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException
            else:
                print("Modo inválido.")
    except RestartGameException:
        print("Mudando modo...")
        main()
    except QuitGameException:
        print("Programa encerrado.")
    except EOFError:
        print("Entrada finalizada.")

if __name__ == "__main__":
    main()

## 4. Resultados

Nesta seção, apresentamos os resultados numéricos e visuais obtidos a partir das simulações de Connect Four e do teste com o conjunto Iris.  
Mostraremos métricas quantitativas, detalhes de partidas de exemplo (“uso ao vivo”) e visualizações da árvore de decisão.

### 4.1. Demonstrações ao Vivo

#### __Jogador Contra MCTS__

In [15]:
%load_ext autoreload
%autoreload 3
from connected_four import ConnectState, GameMeta
from mcts import MCTS
state = ConnectState()
mcts = MCTS(state)
state.to_play = 1
state.move(0) #Jogador joga peça na coluna 0
state.print()
mcts.search(1) # MCTS "pensou" por 1 segundo
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()
state.move(1) #Jogador joga peça na coluna 1
state.print()   
mcts.search(1)
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X |   |   |   |   |   |   |
O MCTS escolheu: 2
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X |   | O |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X | X | O |   |   |   |   |
O MCTS escolheu: 2
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   | O |   |   |   |   |
| X | X | O |   |   |   |   |


__Sequência:__

Jogador (X): 0  
MCTS (O): 2  
Jogador (X): 1  
MCTS (O): 2  

O jogador (X) tenta formar uma linha horizontal nas colunas 0 e 1. O MCTS responde com uma jogada na coluna 2 — interrompendo o avanço e ocupando espaço central estratégico. O facto do MCTS repetir a jogada na mesma coluna (2) mostra que ele reconhece o valor tático da posição (tanto defensivo como central).

#### __Jogador Contra Árvore__

In [11]:
%load_ext autoreload
%autoreload 3
from connected_four import ConnectState, GameMeta
from decision_tree_builder import predict
from game  import state_to_features
import pickle
# carregar árvore de decisão
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)
# Célula de código: executa algumas jogadas
state = ConnectState()
state.to_play = 1
state.move(0) # Jogador (X) faz jogada na coluna 0
state.print()
# Árvore de Decisão responde
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()
state.move(1) # Jogador (X) faz jogada na coluna 1
state.print()
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()
state.move(2) # Jogador (X) faz jogada na coluna 2
state.print()
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X |   |   |   |   |   |   |
Árvore escolheu: 6
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X |   |   |   |   |   | O |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| X | X |   |   |   |   | O |
Árvore escolheu: 6
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | O |
| X | X |   |   |   |   | O |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | O |
| X 

__Sequência de jogadas:__

Jogador (X): 0  
Árvore (O): 3  
Jogador (X): 1  
Árvore (O): 6  
Jogador (X): 2  
Árvore (O): 6  

O jogador (X) tenta formar uma linha de 4 horizontal começando pelas colunas 0–2. A árvore, percebendo a ameaça, joga na coluna 3 e mais tarde repete na 6, demonstrando um comportamento defensivo, bloqueando uma potencial vitória, e consistente. Mostra, assim, que a árvore tem capacidade de bloqueio, mesmo sem usar lookahead profundo como o MCTS.

#### __MCTS contra Árvore__

In [24]:
%load_ext autoreload
%autoreload 3
from connected_four import ConnectState, GameMeta
from decision_tree_builder import predict
from game  import state_to_features
from mcts import MCTS
import pickle
# carregar árvore de decisão
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)
# Célula de código: executa algumas jogadas
state = ConnectState()
mcts = MCTS(state)
mcts.search(4)
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()
# Árvore de Decisão responde
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()
mcts.search(4)
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()
mcts.search(4)
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()
mv = predict(decision_tree, state_to_features(state))
print(f"Árvore escolheu: {mv}")
state.move(mv)
state.print()
mcts.search(4)
print(f"O MCTS escolheu: {mcts.best_move()}")
state.move(mcts.best_move())
state.print()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
O MCTS escolheu: 2
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | X |
Árvore escolheu: 6
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | O |
|   |   |   |   |   |   | X |
O MCTS escolheu: 6
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | O |
|   |   |   |   | X |   | X |
Árvore escolheu: 5
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   | O |
|   |   |   |   | X | O | X |
O MCTS escolheu: 3
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |  

__Sequência de jogadas:__
MCTS (X): 3  
Árvore (O): 2  
MCTS (X): 4  
Árvore (O): 5  
MCTS (X): 6  
Árvore (O): 6

Neste cenário, o MCTS começa pressionando pelo centro e cria uma ameaça diagonal. A árvore responde com jogadas defensivas e ocupa espaços laterais. Apesar de poucas jogadas, já se observa um padrão estratégico emergente de ambos os lados. 

### 4.2. Desempenho do MCTS

O desempenho do algoritmo Monte Carlo Tree Search ode ser avaliado sob três aspectos principais: 
- velocidade de simulações (rollouts);
- tempo médio por decisão;
- qualidade do jogo (win‑rate).

#### __Rollouts por segundo__

Número de simulações completas (“playouts”) que o MCTS executa por segundo de CPU. Cada rollout percorre Seleção → Expansão → Simulação → Retropropagação até o fim do jogo. Mais rollouts geralmente significam melhor estimativa de valor de cada movimento, pois exploram mais cenários futuros.

In [ ]:
import time
mcts = MCTS(ConnectState())
start = time.process_time()
mcts.search(time_limit=1.0)      # 1 segundo de CPU
rollouts = mcts.num_rollouts     # armazenado internamente
elapsed = time.process_time() - start
print(f"{rollouts} rollouts em {elapsed:.2f}s")

#### __Tempo Médio por Decisão__

Tempo de CPU gasto em média para escolher uma jogada. Se usarmos um limite fixo de X segundos em cada iteração de search, este será próximo de X, mas pequenas variações ocorrem por conta do overhead de Python e estruturas de dados. Ajuda a entender se o algoritmo é viável em tempo real. Por exemplo, para um agente adversário rápido, 3 s por jogada pode ser aceitável; para um servidor de partidas online com mil partidas simultâneas, talvez não.

In [ ]:
times = []
for _ in range(10):
    t0 = time.process_time()
    mcts.search(time_limit=3.0)   # 3s solicitados
    times.append(time.process_time() - t0)
print("Média por move:", sum(times)/len(times), "s")

#### __Qualidade de Jogo (Win-Rate)__

Proporção de partidas em que o MCTS sai vencedor contra outro jogador (pode ser contra si mesmo ou outra pessoa, contra a árvore de decisão ou contra o MCTS).
Como medir:
- MCTS vs MCTS: fixar ambos com mesmo tempo por jogada (por ex. 5 s) e jogar N partidas, alternando quem começa:

In [43]:
from game import play_ai_vs_ai #não usado no jogo, definido no ficheiro game.py
from mcts import MCTS, Node
N = 3
wins1 = 0
wins2 = 0
for _ in range(N):
    state1 = ConnectState()
    state2 = ConnectState()
    mcts1 = MCTS(state1)
    mcts2 = MCTS(state2)
    outcome = play_ai_vs_ai(mcts1)
    if outcome == 1:
        wins1 += 1
        print("Jogador 1 venceu!")
    elif outcome == 2:
        wins2 += 1
        print("Jogador 2 venceu!")
    else:
        print("Empate!")
win_rate1 = wins1 / N
win_rate2 = wins2 / N
print(f"Taxa de vitórias do Jogador 1: {win_rate1:.2%}")
print(f"Taxa de vitórias do Jogador 2: {win_rate2:.2%}")

|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
MCTS O a pensar...
MCTS O escolheu a coluna: 2
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
MCTS X a pensar...
MCTS X escolheu a coluna: 1
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   | X |   |   |   |   |   |
MCTS O a pensar...
MCTS O escolheu a coluna: 6
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   | X |   |   |   |   | O |
MCTS X a pensar...
MCTS X escolheu a coluna: 5
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
| 

- MCTS vs Árvore de Decisão: MCTS com X s/jogada vs árvore que responde instantaneamente (predict):

In [ ]:
%load_ext autoreload
%autoreload 2
from mcts import MCTS, Node
from game import play_ai_vs_tree
N = 3
MCTS_PLAYER = 1
wins = 0
for _ in range(N):
    outcome = play_ai_vs_tree(starting_player=1, tree=decision_tree)
    if outcome == MCTS_PLAYER: wins += 1
win_rate = wins / N
print(f"Taxa de vitória do MCTS: {win_rate:.2%}")

In [ ]:
import time
import matplotlib.pyplot as plt

def calcular_accuracy_id3(model, X_test, y_test):
    """
    Recebe um modelo ID3 treinado, dados de teste X_test e rótulos y_test,
    retorna a acurácia em porcentagem (implementação pura em Python).
    """
    y_pred = model.predict(X_test)
    correct = sum(1 for yt, yp in zip(y_test, y_pred) if yt == yp)
    acc = (correct / len(y_test)) * 100
    return acc


def calcular_tempo_medio_id3(model, X):
    """
    Recebe um modelo ID3 e um conjunto de entradas X (iterável),
    retorna o tempo médio (em segundos) de predição por amostra.
    """
    start = time.perf_counter()
    for x in X:
        model.predict([x])
    end = time.perf_counter()
    total = end - start
    return total / len(X)


def calcular_win_rate_id3(model, adversario, partidas, get_estado_inicial, realizar_jogada):
    """
    Executa um número de partidas onde o ID3 joga contra o adversário.
    - model: árvore ID3 treinada (joga como 'player1')
    - adversario: função ou objeto que, dado o estado, retorna jogada
    - partidas: número de partidas a simular
    - get_estado_inicial: função que retorna estado inicial do jogo
    - realizar_jogada: função que aplica jogada no estado e retorna novo estado e vencedor
    Retorna porcentual de vitórias do ID3.
    """
    vitorias = 0
    for _ in range(partidas):
        estado = get_estado_inicial()
        atual = 'id3'
        vencedor = None
        while vencedor is None:
            if atual == 'id3':
                jogada = model.predict([state_to_features(estado)])[0]
            else:
                jogada = adversario(estado)
            estado, vencedor = realizar_jogada(estado, jogada, atual)
            atual = 'adversario' if atual == 'id3' else 'id3'
        if vencedor == 'id3':
            vitorias += 1
    return (vitorias / partidas) * 100

# Exemplo de curva ID3: acurácia vs profundidade
depths = [2,4,6,8,10]
acc = [72.5, 75.9, 78.2, 77.4, 76.0]  # seus dados aqui
plt.plot(depths, acc, marker='o')
plt.xlabel('Profundidade Máxima')
plt.ylabel('Acurácia (%)')
plt.title('Sensibilidade da Acurácia à Profundidade (ID3)')
plt.show()

## 5. Discussão dos Resultados

A taxa média de rollouts por segundo foi de 77, o que, embora modesto, revelou-se suficiente para tomadas de decisão eficazes no contexto do Connect Four, como demonstrado nos exemplos apresentados anteriormente.

Média por move: 3.0102835999999997 s

MCTS vs Arvore: Taxa de vitória do MCTS: 100.00%

MCTS x MCTS: 
- Taxa de vitórias do Jogador 1: 100.00%
- Taxa de vitórias do Jogador 2: 0.00%
O jogador 1 começou sempre primeiro, logo tem a vantagem. Mesmo assim, preencheram quase todo o tabuleiro em todos os jogos de teste.

![Accuracy x Depth](acuráciaxprofundidade.png)

- Aumento inicial de 2 → 6: a acurácia sobe de ~72.5% até ~78.2% em profundidade 6, indicando que a árvore está colhendo mais padrões relevantes.
- Queda após profundidade 6: a partir daqui (8 e 10) a acurácia diminui (≈77.4% e 76.0%), sinal clássico de overfitting — o modelo começa a “decorar” ruídos do conjunto de treino e perde capacidade de generalizar.

Forças do ID3

Interpretabilidade: árvore fácil de visualizar e entender.

Rápida inferência: predição em poucos microssegundos.

Fraquezas do ID3

Overfitting se profundidade excessiva.

Sensível à discretização: escolhas de bins podem distorcer padrões.

Forças do MCTS

Adaptativo: alocação dinâmica de rollouts onde importa.

Robustez em espaços grandes de jogo.

Fraquezas do MCTS

Custo computacional: tempo e memória para muitos rollouts.

Requer calibração de  para bom desempenho.

Comparação geral:

Cenários simples (board estável, poucos movimentos): ID3 pode competir bem.

Cenários complexos (alto branching factor): MCTS tende a superar.



## 6. Conclusão 

## 7. Referências

Qi Wang’s Blog! (2022). Connect 4 with Monte Carlo Tree Search. [online] Available at: https://www.harrycodes.com/blog/monte-carlo-tree-search.
‌